In [1]:
!pip install pyod

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyod: filename=pyod-1.0.9-py3-none-any.whl size=184112 sha256=f43a3c3f9cb47d91a31bc7c87bcbde6be84653c6169fe9525dbadcfea38ef540
  Stored in directory: /root/.cache/pip/wheels/1b/9c/b8/9759d7cc64a1e01bb9872ade80cb7db445ccf506e083325106
Successfully built pyod


In [21]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np

from pyod.models.ecod import ECOD

In [22]:
def calculate_metrics(y_true, y_pred):
  # Compute true positives, true negatives, false positives, and false negatives
  tp = sum([1 for i in range(len(y_true)) if y_true[i] == 1 and y_pred[i] == 1])
  tn = sum([1 for i in range(len(y_true)) if y_true[i] == 0 and y_pred[i] == 0])
  fp = sum([1 for i in range(len(y_true)) if y_true[i] == 0 and y_pred[i] == 1])
  fn = sum([1 for i in range(len(y_true)) if y_true[i] == 1 and y_pred[i] == 0])

  # Compute accuracy, recall, and precision
  accuracy = (tp + tn) / (tp + tn + fp + fn)
  recall = tp / (tp + fn)
  precision = tp / (tp + fp)
  
  return accuracy, recall, precision

# In this section, we define Resnet Class

In [23]:
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=100):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        
        out = self.linear(out)
        return out


def ResNet18(num_classes):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes)


def load_model(model_path, num_classes):
    model = ResNet18(num_classes)
    ckpt = torch.load(model_path, map_location ='cpu')
    model.load_state_dict(ckpt)
    model.cuda()
    model.eval()
    return model

# Loading CIFAR Dataset, Adverseial Data, and Trained ResNet model

In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## CIFAR10 Experiment:

Loading trained model on CIFAR10

In [25]:
model_path = '/content/model-ResNet18-cifar10-best_epoch.pt'

num_classes = 10

model_cifar10 = load_model(model_path, num_classes)
model_cifar10.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

Loading adverseial data smaples

In [27]:
attacked_data_cifar10_L2 = torch.load("/content/CIFAR10_L2_aa_standard_1_1000_eps_0.03137.pth")
attacked_data_cifar10_L2_values = attacked_data_cifar10_L2['adv_complete']

In [28]:
attacked_data_cifar10_Linf = torch.load("/content/CIFAR10_Linf_aa_standard_1_1000_eps_0.03137.pth")
attacked_data_cifar10_Linf_values = attacked_data_cifar10_Linf['adv_complete']

Loading CIFAR10

In [29]:
from keras.datasets import cifar10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

### Anomaly Detection on CIFAR10 and with L2 norm on Autoattack

In [30]:
model_cifar10.to(device)
attacked_data_values = attacked_data_cifar10_L2_values.to(device)

Get class prediction for attacked data

In [31]:
output_model = model_cifar10(attacked_data_values)
class_predicted = torch.argmax(output_model, dim=1)

Feature Extraction and Anomaly Detection

In [32]:
#Create empty arrays to store the true labels (y_all_classes) and predicted labels (y_pred_all_classes) for all classes in the dataset.
y_all_classes = np.array([])
y_pred_all_classes = np.array([])

#Loop through each class in the dataset.
for class_label in range(num_classes):
# Get indices of training set examples for class 0 (airplane)
  indices = np.where(train_labels == class_label)[0]

  # Extract training set examples for class 0
  # Normal data
  train_x = train_images[indices]
  train_y = np.array([0] * len(train_x))

  #Attacked Sample
  indices_attacked = np.where(class_predicted.cpu() == class_label)
  if len(indices_attacked[0]) > 0:
    attacked_x = attacked_data_values[indices_attacked].cpu().numpy()
    attacked_x = np.transpose(attacked_x, (0, 2, 3, 1))
    attacked_y = np.array([1] * len(attacked_x))

    """

    Load a pre-trained ResNet model and use it to extract features from the normal and attacked data.
    The features are concatenated to form a new dataset (x_all_feature) and used to train an ECOD detector.

    """
    model = load_model(model_path, num_classes)

    # Extract feature from normal data using our ResNet Model
    all_out = []
    for i in range(int(np.floor(len(train_x)/64))):
      input_model = train_x[i*64:(i+1)*64].transpose((0, 3, 1, 2))
      input_model = torch.from_numpy(input_model)
      input_model = input_model.to(device).float()
      out = model.conv1(input_model)
      out = model.bn1(out)
      out = model.layer1(out)
      out = model.layer2(out)
      out = model.layer3(out)
      out = model.layer4(out)
      out = F.avg_pool2d(out, 4)
      out = out.view(out.size(0), -1)
      out = list(out.cpu().detach().numpy())
      all_out = all_out + out
    all_out = np.array(all_out)


    # Extract feature from attack data using our ResNet Model
    all_out_attack = []
    ran = max(1,int(np.floor(len(attacked_x)/64)))
    for i in range(ran):
      input_model = attacked_x[i*64:(i+1)*64].transpose((0, 3, 1, 2))
      input_model = torch.from_numpy(input_model)
      input_model = input_model.to(device).float()
      out = model.conv1(input_model)
      out = model.bn1(out)
      out = model.layer1(out)
      out = model.layer2(out)
      out = model.layer3(out)
      out = model.layer4(out)
      out = F.avg_pool2d(out, 4)
      out = out.view(out.size(0), -1)
      out = list(out.cpu().detach().numpy())
      all_out_attack = all_out_attack + out
    all_out_attack = np.array(all_out_attack)


    # Concatentate Normal and Attacked data
    x_all_feature = np.concatenate((all_out, all_out_attack), axis=0)
    y_all = np.concatenate((train_y[0:len(all_out)], attacked_y[0:len(all_out_attack)]), axis=0)


    # train ECOD detector
    clf_name = 'ECOD'
    clf = ECOD()

    # you could try parallel version as well.
    # clf = ECOD(n_jobs=2)
    clf.fit(x_all_feature)

    # get the prediction labels and outlier scores of the training data
    # y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
    y_train_scores = clf.decision_scores_  # raw outlier scores

    y_train_pred = y_train_scores > y_train_scores.mean() + 3.5* y_train_scores.std()


    y_pred_all_classes = np.concatenate((y_pred_all_classes, y_train_pred), axis=0)
    y_all_classes = np.concatenate((y_all_classes, y_all), axis=0)

    try:
      accuracy, recall, precision = calculate_metrics(y_train_pred, y_all)
      print("*********************************************")
      print("Class label:", class_label)
      print("Accuracy:", accuracy)
      print("Recall:", recall)
      print("Precision:", precision)
    except:
      print(class_label)

*********************************************
Class label: 0
Accuracy: 0.9970332278481012
Recall: 0.810126582278481
Precision: 1.0
*********************************************
Class label: 1
Accuracy: 0.9984177215189873
Recall: 0.8888888888888888
Precision: 1.0
*********************************************
Class label: 2
Accuracy: 0.9996044303797469
Recall: 0.9696969696969697
Precision: 1.0
*********************************************
Class label: 3
Accuracy: 0.9996044303797469
Recall: 0.9696969696969697
Precision: 1.0
*********************************************
Class label: 4
Accuracy: 0.9994066455696202
Recall: 0.9552238805970149
Precision: 1.0
*********************************************
Class label: 5
Accuracy: 0.9982199367088608
Recall: 0.8767123287671232
Precision: 1.0
*********************************************
Class label: 6
Accuracy: 0.9992088607594937
Recall: 0.9411764705882353
Precision: 1.0
*********************************************
Class label: 7
Accuracy: 0.9970

In [33]:
accuracy, recall, precision = calculate_metrics(y_all_classes, y_pred_all_classes)
print("******************Cifar10-L2***************************")
print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)

******************Cifar10-L2***************************
Accuracy: 0.9986155063291139
Recall: 1.0
Precision: 0.9014084507042254


### Anomaly Detection on CIFAR10 and with Linf norm on Autoattack

In [34]:
model_cifar10.to(device)
attacked_data_values = attacked_data_cifar10_Linf_values.to(device)

Get class prediction for attacked data

In [35]:
output_model = model_cifar10(attacked_data_values)
class_predicted = torch.argmax(output_model, dim=1)

Feature Extraction and Anomaly Detection

In [36]:
#Create empty arrays to store the true labels (y_all_classes) and predicted labels (y_pred_all_classes) for all classes in the dataset.
y_all_classes = np.array([])
y_pred_all_classes = np.array([])

#Loop through each class in the dataset.
for class_label in range(num_classes):
# Get indices of training set examples for class 0 (airplane)
  indices = np.where(train_labels == class_label)[0]

  # Extract training set examples for class 0
  # Normal data
  train_x = train_images[indices]
  train_y = np.array([0] * len(train_x))

  #Attacked Sample
  indices_attacked = np.where(class_predicted.cpu() == class_label)
  if len(indices_attacked[0]) > 0:
    attacked_x = attacked_data_values[indices_attacked].cpu().numpy()
    attacked_x = np.transpose(attacked_x, (0, 2, 3, 1))
    attacked_y = np.array([1] * len(attacked_x))

    """

    Load a pre-trained ResNet model and use it to extract features from the normal and attacked data.
    The features are concatenated to form a new dataset (x_all_feature) and used to train an ECOD detector.

    """
    model = load_model(model_path, num_classes)

    # Extract feature from normal data using our ResNet Model
    all_out = []
    for i in range(int(np.floor(len(train_x)/64))):
      input_model = train_x[i*64:(i+1)*64].transpose((0, 3, 1, 2))
      input_model = torch.from_numpy(input_model)
      input_model = input_model.to(device).float()
      out = model.conv1(input_model)
      out = model.bn1(out)
      out = model.layer1(out)
      out = model.layer2(out)
      out = model.layer3(out)
      out = model.layer4(out)
      out = F.avg_pool2d(out, 4)
      out = out.view(out.size(0), -1)
      out = list(out.cpu().detach().numpy())
      all_out = all_out + out
    all_out = np.array(all_out)


    # Extract feature from attack data using our ResNet Model
    all_out_attack = []
    ran = max(1,int(np.floor(len(attacked_x)/64)))
    for i in range(ran):
      input_model = attacked_x[i*64:(i+1)*64].transpose((0, 3, 1, 2))
      input_model = torch.from_numpy(input_model)
      input_model = input_model.to(device).float()
      out = model.conv1(input_model)
      out = model.bn1(out)
      out = model.layer1(out)
      out = model.layer2(out)
      out = model.layer3(out)
      out = model.layer4(out)
      out = F.avg_pool2d(out, 4)
      out = out.view(out.size(0), -1)
      out = list(out.cpu().detach().numpy())
      all_out_attack = all_out_attack + out
    all_out_attack = np.array(all_out_attack)


    # Concatentate Normal and Attacked data
    x_all_feature = np.concatenate((all_out, all_out_attack), axis=0)
    y_all = np.concatenate((train_y[0:len(all_out)], attacked_y[0:len(all_out_attack)]), axis=0)


    # train ECOD detector
    clf_name = 'ECOD'
    clf = ECOD()

    # you could try parallel version as well.
    # clf = ECOD(n_jobs=2)
    clf.fit(x_all_feature)

    # get the prediction labels and outlier scores of the training data
    # y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
    y_train_scores = clf.decision_scores_  # raw outlier scores

    y_train_pred = y_train_scores > y_train_scores.mean() + 3.5* y_train_scores.std()


    y_pred_all_classes = np.concatenate((y_pred_all_classes, y_train_pred), axis=0)
    y_all_classes = np.concatenate((y_all_classes, y_all), axis=0)

    try:
      accuracy, recall, precision = calculate_metrics(y_train_pred, y_all)
      print("*********************************************")
      print("Class label:", class_label)
      print("Accuracy:", accuracy)
      print("Recall:", recall)
      print("Precision:", precision)
    except:
      print(class_label)

*********************************************
Class label: 0
Accuracy: 0.9970332278481012
Recall: 0.810126582278481
Precision: 1.0
*********************************************
Class label: 1
Accuracy: 0.9972150387905311
Recall: 0.7142857142857143
Precision: 1.0
*********************************************
Class label: 2
Accuracy: 0.9653201219512195
Recall: 0.9743589743589743
Precision: 0.296875
*********************************************
Class label: 3
Accuracy: 0.9994066455696202
Recall: 0.9552238805970149
Precision: 1.0
*********************************************
Class label: 4
Accuracy: 0.9994066455696202
Recall: 0.9552238805970149
Precision: 1.0
*********************************************
Class label: 5
Accuracy: 0.9978131212723658
Recall: 0.7755102040816326
Precision: 1.0
*********************************************
Class label: 6
Accuracy: 0.9934413580246914
Recall: 1.0
Precision: 0.8229166666666666
*********************************************
Class label: 7
Accuracy: 0

In [37]:
accuracy, recall, precision = calculate_metrics(y_all_classes, y_pred_all_classes)
print("******************Cifar10-Linf***************************")
print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)

******************Cifar10-Linf***************************
Accuracy: 0.9937749935976992
Recall: 0.7461447212336892
Precision: 0.8604651162790697


## CIFAR100 Experiment:

Loading trained model on CIFAR100

In [38]:
model_path = '/content/model-ResNet18-cifar100-best_epoch.pt'

num_classes = 100


model_cifar100 = load_model(model_path, num_classes)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

Loading adverseial data smaples

In [39]:
attacked_data_cifar100_L2 = torch.load("/content/CIFAR100_L2_aa_standard_1_1000_eps_0.03137.pth")
attacked_data_cifar100_L2_values = attacked_data_cifar100_L2['adv_complete']

In [40]:
attacked_data_cifar100_Linf = torch.load("/content/CIFAR100_Linf_aa_standard_1_1000_eps_0.03137.pth")
attacked_data_cifar100_Linf_values = attacked_data_cifar100_Linf['adv_complete']

Loading CIFAR100

In [41]:
from keras.datasets import cifar100
(train_images, train_labels), (test_images, test_labels) = cifar100.load_data()

### Anomaly Detection on CIFAR100 and with L2 norm on Autoattack

In [42]:
model_cifar100.to(device)
attacked_data_values = attacked_data_cifar100_L2_values.to(device)

Get class prediction for attacked data

In [43]:
output_model = model_cifar100(attacked_data_values)
class_predicted = torch.argmax(output_model, dim=1)

Feature Extraction and Anomaly Detection

In [44]:
#Create empty arrays to store the true labels (y_all_classes) and predicted labels (y_pred_all_classes) for all classes in the dataset.
y_all_classes = np.array([])
y_pred_all_classes = np.array([])

#Loop through each class in the dataset.
for class_label in range(num_classes):
# Get indices of training set examples for class 0 (airplane)
  indices = np.where(train_labels == class_label)[0]

  # Extract training set examples for class 0
  # Normal data
  train_x = train_images[indices]
  train_y = np.array([0] * len(train_x))

  #Attacked Sample
  indices_attacked = np.where(class_predicted.cpu() == class_label)
  if len(indices_attacked[0]) > 0:
    attacked_x = attacked_data_values[indices_attacked].cpu().numpy()
    attacked_x = np.transpose(attacked_x, (0, 2, 3, 1))
    attacked_y = np.array([1] * len(attacked_x))



    """

    Load a pre-trained ResNet model and use it to extract features from the normal and attacked data.
    The features are concatenated to form a new dataset (x_all_feature) and used to train an ECOD detector.

    """
    model = load_model(model_path, num_classes)

    # Extract feature from normal data using our ResNet Model
    all_out = []
    for i in range(int(np.floor(len(train_x)/64))):
      input_model = train_x[i*64:(i+1)*64].transpose((0, 3, 1, 2))
      input_model = torch.from_numpy(input_model)
      input_model = input_model.to(device).float()
      out = model.conv1(input_model)
      out = model.bn1(out)
      out = model.layer1(out)
      out = model.layer2(out)
      out = model.layer3(out)
      out = model.layer4(out)
      out = F.avg_pool2d(out, 4)
      out = out.view(out.size(0), -1)
      out = list(out.cpu().detach().numpy())
      all_out = all_out + out
    all_out = np.array(all_out)


    # Extract feature from attack data using our ResNet Model
    all_out_attack = []
    ran = max(1,int(np.floor(len(attacked_x)/64)))
    for i in range(ran):
      input_model = attacked_x[i*64:(i+1)*64].transpose((0, 3, 1, 2))
      input_model = torch.from_numpy(input_model)
      input_model = input_model.to(device).float()
      out = model.conv1(input_model)
      out = model.bn1(out)
      out = model.layer1(out)
      out = model.layer2(out)
      out = model.layer3(out)
      out = model.layer4(out)
      out = F.avg_pool2d(out, 4)
      out = out.view(out.size(0), -1)
      out = list(out.cpu().detach().numpy())
      all_out_attack = all_out_attack + out
    all_out_attack = np.array(all_out_attack)


    # Concatentate Normal and Attacked data
    x_all_feature = np.concatenate((all_out, all_out_attack), axis=0)
    y_all = np.concatenate((train_y[0:len(all_out)], attacked_y[0:len(all_out_attack)]), axis=0)


    # train ECOD detector
    clf_name = 'ECOD'
    clf = ECOD()

    # you could try parallel version as well.
    # clf = ECOD(n_jobs=2)
    clf.fit(x_all_feature)

    # get the prediction labels and outlier scores of the training data
    # y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
    y_train_scores = clf.decision_scores_  # raw outlier scores

    y_train_pred = y_train_scores > y_train_scores.mean() + 3.5* y_train_scores.std()


    y_pred_all_classes = np.concatenate((y_pred_all_classes, y_train_pred), axis=0)
    y_all_classes = np.concatenate((y_all_classes, y_all), axis=0)

    try:
      accuracy, recall, precision = calculate_metrics(y_train_pred, y_all)
      print("*********************************************")
      print("Class label:", class_label)
      print("Accuracy:", accuracy)
      print("Recall:", recall)
      print("Precision:", precision)
    except:
      print(class_label)

*********************************************
Class label: 0
Accuracy: 1.0
Recall: 1.0
Precision: 1.0
*********************************************
Class label: 1
Accuracy: 1.0
Recall: 1.0
Precision: 1.0
*********************************************
Class label: 2
Accuracy: 1.0
Recall: 1.0
Precision: 1.0
*********************************************
Class label: 3
Accuracy: 1.0
Recall: 1.0
Precision: 1.0
*********************************************
Class label: 4
Accuracy: 1.0
Recall: 1.0
Precision: 1.0
*********************************************
Class label: 5
Accuracy: 1.0
Recall: 1.0
Precision: 1.0
*********************************************
Class label: 6
Accuracy: 1.0
Recall: 1.0
Precision: 1.0
*********************************************
Class label: 7
Accuracy: 0.9978401727861771
Recall: 1.0
Precision: 0.9333333333333333
*********************************************
Class label: 8
Accuracy: 1.0
Recall: 1.0
Precision: 1.0
*********************************************
Class 

In [45]:
accuracy, recall, precision = calculate_metrics(y_all_classes, y_pred_all_classes)
print("******************Cifar100-L2***************************")
print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)

******************Cifar100-L2***************************
Accuracy: 0.9999126637554585
Recall: 0.996
Precision: 1.0


### Anomaly Detection on CIFAR100 and with Linf norm on Autoattack

In [46]:
model_cifar100.to(device)
attacked_data_values = attacked_data_cifar100_Linf_values.to(device)

Get class prediction for attacked data

In [47]:
output_model = model_cifar100(attacked_data_values)
class_predicted = torch.argmax(output_model, dim=1)

Feature Extraction and Anomaly Detection

In [48]:
#Create empty arrays to store the true labels (y_all_classes) and predicted labels (y_pred_all_classes) for all classes in the dataset.
y_all_classes = np.array([])
y_pred_all_classes = np.array([])

#Loop through each class in the dataset.
for class_label in range(num_classes):
# Get indices of training set examples for class 0 (airplane)
  indices = np.where(train_labels == class_label)[0]

  # Extract training set examples for class 0
  # Normal data
  train_x = train_images[indices]
  train_y = np.array([0] * len(train_x))

  #Attacked Sample
  indices_attacked = np.where(class_predicted.cpu() == class_label)
  if len(indices_attacked[0]) > 0:
    attacked_x = attacked_data_values[indices_attacked].cpu().numpy()
    attacked_x = np.transpose(attacked_x, (0, 2, 3, 1))
    attacked_y = np.array([1] * len(attacked_x))



    """

    Load a pre-trained ResNet model and use it to extract features from the normal and attacked data.
    The features are concatenated to form a new dataset (x_all_feature) and used to train an ECOD detector.

    """
    model = load_model(model_path, num_classes)

    # Extract feature from normal data using our ResNet Model
    all_out = []
    for i in range(int(np.floor(len(train_x)/64))):
      input_model = train_x[i*64:(i+1)*64].transpose((0, 3, 1, 2))
      input_model = torch.from_numpy(input_model)
      input_model = input_model.to(device).float()
      out = model.conv1(input_model)
      out = model.bn1(out)
      out = model.layer1(out)
      out = model.layer2(out)
      out = model.layer3(out)
      out = model.layer4(out)
      out = F.avg_pool2d(out, 4)
      out = out.view(out.size(0), -1)
      out = list(out.cpu().detach().numpy())
      all_out = all_out + out
    all_out = np.array(all_out)


    # Extract feature from attack data using our ResNet Model
    all_out_attack = []
    ran = max(1,int(np.floor(len(attacked_x)/64)))
    for i in range(ran):
      input_model = attacked_x[i*64:(i+1)*64].transpose((0, 3, 1, 2))
      input_model = torch.from_numpy(input_model)
      input_model = input_model.to(device).float()
      out = model.conv1(input_model)
      out = model.bn1(out)
      out = model.layer1(out)
      out = model.layer2(out)
      out = model.layer3(out)
      out = model.layer4(out)
      out = F.avg_pool2d(out, 4)
      out = out.view(out.size(0), -1)
      out = list(out.cpu().detach().numpy())
      all_out_attack = all_out_attack + out
    all_out_attack = np.array(all_out_attack)


    # Concatentate Normal and Attacked data
    x_all_feature = np.concatenate((all_out, all_out_attack), axis=0)
    y_all = np.concatenate((train_y[0:len(all_out)], attacked_y[0:len(all_out_attack)]), axis=0)


    # train ECOD detector
    clf_name = 'ECOD'
    clf = ECOD()

    # you could try parallel version as well.
    # clf = ECOD(n_jobs=2)
    clf.fit(x_all_feature)

    # get the prediction labels and outlier scores of the training data
    # y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
    y_train_scores = clf.decision_scores_  # raw outlier scores

    y_train_pred = y_train_scores > y_train_scores.mean() + 3.5* y_train_scores.std()


    y_pred_all_classes = np.concatenate((y_pred_all_classes, y_train_pred), axis=0)
    y_all_classes = np.concatenate((y_all_classes, y_all), axis=0)

    try:
      accuracy, recall, precision = calculate_metrics(y_train_pred, y_all)
      print("*********************************************")
      print("Class label:", class_label)
      print("Accuracy:", accuracy)
      print("Recall:", recall)
      print("Precision:", precision)
    except:
      print(class_label)

*********************************************
Class label: 0
Accuracy: 1.0
Recall: 1.0
Precision: 1.0
*********************************************
Class label: 1
Accuracy: 1.0
Recall: 1.0
Precision: 1.0
*********************************************
Class label: 2
Accuracy: 1.0
Recall: 1.0
Precision: 1.0
*********************************************
Class label: 3
Accuracy: 1.0
Recall: 1.0
Precision: 1.0
*********************************************
Class label: 4
Accuracy: 0.9978902953586498
Recall: 1.0
Precision: 0.9615384615384616
*********************************************
Class label: 5
Accuracy: 1.0
Recall: 1.0
Precision: 1.0
*********************************************
Class label: 6
Accuracy: 1.0
Recall: 1.0
Precision: 1.0
*********************************************
Class label: 7
Accuracy: 1.0
Recall: 1.0
Precision: 1.0
*********************************************
Class label: 8
Accuracy: 1.0
Recall: 1.0
Precision: 1.0
*********************************************
Class 

In [49]:
accuracy, recall, precision = calculate_metrics(y_all_classes, y_pred_all_classes)
print("******************Cifar100-Linf***************************")
print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)

******************Cifar100-Linf***************************
Accuracy: 0.9956583357703642
Recall: 0.8074222668004012
Precision: 0.9987593052109182
